In [1]:
import duckdb
import ujson as json
import pandas as pd
from rich.console import Console

In [2]:

with open('../jace/.cache/jace_all_cards.json', 'r', encoding='utf-8') as f:
    data = json.loads(f.read())

all_cards_df = pd.DataFrame(data)
all_cards_df.head(10)

,object,id,oracle_id,multiverse_ids,mtgo_id,arena_id,tcgplayer_id,name,lang,released_at,...,preview,color_indicator,tcgplayer_etched_id,content_warning,flavor_name,attraction_lights,variation_of,life_modifier,hand_modifier,defense
0,card,0000419b-0bba-4488-8f7a-6194544ce91e,b34bb2dc-c1af-4d77-b0b3-a0fb342a5fc6,[668564],129825.0,91829.0,558404.0,Forest,en,2024-08-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,NaN,14240.0,Fury Sliver,en,2006-10-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,NaN,33347.0,Kor Outfitter,en,2009-10-02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,00009878-d086-46f0-a964-15734d8368ac,30cd69a8-7893-4075-94ca-04450ff821d3,[433932],NaN,NaN,NaN,Spirit of the Hearth,fr,2017-08-25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,Spirit,en,2015-05-22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,66119.0,145764.0,Siren Lookout,en,2017-09-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,card,00011897-9c8b-482f-8d64-9f2cd8403b6a,d37c4e77-5023-4d13-9cfe-a43f513364bf,[169549],NaN,NaN,NaN,Wormfang Drake,pt,2002-05-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,card,00012bd8-ed68-4978-a22d-f450c8a6e048,5aa12aff-db3c-4be5-822b-3afdf536b33e,[1278],NaN,NaN,1623.0,Web,en,1994-04-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,card,00014127-a9d2-4e01-ae79-2349e8989793,ae2c89c2-66d6-484a-a176-9ea19e00f445,[420538],NaN,NaN,NaN,"Chandra, Pyrogenius",ja,2016-09-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,card,0001680b-82ac-48aa-9416-f97cacc745ef,f4d54a89-8409-4fbc-b01f-3b03f352820f,[114499],NaN,NaN,NaN,Transluminant,ru,2005-10-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
db = duckdb.connect(':memory:')
db.register('all_cards', all_cards_df)

## Can probably just make this into a view, tbh.
_df = db.sql(f'SELECT * EXCLUDE (uri, scryfall_uri, image_uris, set_uri, set_search_uri, scryfall_set_uri, rulings_uri, prints_search_uri) FROM all_cards;').df()
db.register('trimmed_cards', _df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
import io
from itertools import compress

def info_to_df(d: pd.DataFrame, show_memory: bool = True) -> pd.DataFrame:
    """Creates a dataframe from the df.info() call by capturing the io buffer and then parsing it.
    ## TODO: This will probably blow up if there is a truncation of the text ( e.g. "...." line ).
    :param d: Dataframe to be parsed.
    :param show_memory: Should the memory be printed or not.
    :return: The parsed info() call for the dataframe, as a dataframe.
    """
    buf = io.StringIO()
    d.info(buf=buf, memory_usage=show_memory, verbose=True)
    s = buf.getvalue().strip()

    # We want to skip the main header: <class 'pandas.core.frame.DataFrame'>
    lines = [_.split() for _ in s.splitlines()[1:]]

    ## Get the index of the placeholder text `---` line in the output.
    data_index = next((i + 1 for i, _ in enumerate(lines) if _[0] == '---'), None)
    ## Get the index of the dtypes line
    dtypes_index = next((i for i, _ in enumerate(lines) if _[0] == 'dtypes:'), None)

    lines, detail_lines = lines[data_index:dtypes_index], [*lines[dtypes_index:], *lines[:data_index - 2]]
    new_lines = [
        list(
            compress(sub, [(i != 3 or val != 'non-null') for i, val in enumerate(sub)])
        ) for sub in lines
    ]

    columns=['Id', 'Column', 'NonNullCount', 'Dtype']
    result = pd.DataFrame(new_lines, columns=columns)
    ## I want to keep this detail, but if this was prod code then I wouldn't do it this way.
    result['Details'] = str(detail_lines)
    return result

info_to_df(_df, show_memory=False).head()


,Id,Column,NonNullCount,Dtype,Details
0,0,object,492491,object,"[['dtypes:', 'bool(13),', 'float64(9),', 'obje..."
1,1,id,492491,object,"[['dtypes:', 'bool(13),', 'float64(9),', 'obje..."
2,2,oracle_id,492437,object,"[['dtypes:', 'bool(13),', 'float64(9),', 'obje..."
3,3,multiverse_ids,492491,object,"[['dtypes:', 'bool(13),', 'float64(9),', 'obje..."
4,4,mtgo_id,52644,float64,"[['dtypes:', 'bool(13),', 'float64(9),', 'obje..."


### Notes

1. Each `id` is mapped to a card, the granularity is at least at the language level, the `oracle_id` matches up with the individual card by  name.

In [19]:
price_df = db.sql(f"""
SELECT set_name, set, name, rarity, prices.*, lang, full_art, id, oracle_id, set_id
FROM trimmed_cards
ORDER BY name desc
""").df()

cols_to_fill = ['usd', 'usd_foil', 'usd_etched', 'eur', 'eur_foil']
price_df[cols_to_fill] = price_df[cols_to_fill].fillna(0)
db.register('prices', price_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [35]:
db.sql(f"""
SELECT set_name
    , ROUND(AVG(usd::DECIMAL(18,4)), 2) AS avg_usd
    , ROUND(MIN(usd::DECIMAL(18,4))) AS min_usd
    , ROUND(MAX(usd::DECIMAL(18,4))) AS max_usd
    , ROUND(AVG(usd_foil::DECIMAL(18,4)), 2) AS avg_usd_foil
    , ROUND(AVG(usd_etched::DECIMAL(18,4)), 2) AS avg_usd_etched
FROM prices
WHERE 1=1
GROUP BY set_name
""").df().head(50)

,set_name,avg_usd,min_usd,max_usd,avg_usd_foil,avg_usd_etched
0,Core Set 2020,0.06,0.0,32.0,0.14,0.0
1,Commander 2018,0.09,0.0,10.0,0.01,0.0
2,Ravnica Remastered,0.35,0.0,57.0,4.13,0.0
3,Jumpstart 2022,0.10,0.0,57.0,0.00,0.0
4,30th Anniversary Edition,17.54,0.0,1101.0,0.00,0.0
5,Eventide,0.15,0.0,18.0,0.81,0.0
6,Outlaws of Thunder Junction Commander,0.14,0.0,8.0,0.01,0.0
7,Zendikar Rising Promos,0.65,0.0,9.0,3.40,0.0
8,Chronicles Foreign Black Border,0.00,0.0,0.0,0.00,0.0
9,Tempest Remastered,0.00,0.0,0.0,0.00,0.0
